<a href="https://colab.research.google.com/github/rodrigonevest/projetophyton/blob/master/DataSet_Corn_Cepea.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import urllib3
from bs4 import BeautifulSoup
import pandas as pd
import re
import nltk
import numpy as np
 
 
nltk.download('book')

[nltk_data] Downloading collection 'book'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/chat80.zip.
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package conll2000 to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/conll2000.zip.
[nltk_data]    | Downloading package conll2002 to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/conll2002.zip.
[nltk_data]    | Downloading package dependency_treebank to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/dependency_treebank.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    

True

# Fazendo Web Crawler
|| Extraindo a Data, Link, Machete e Noticias 
| Site : http://soybeansandcorn.com


In [2]:
 
def BuscaTexto(pagina):
  
  urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
 
  #Criando o dataframe para salvar os dados
  df = pd.DataFrame(index=['Date','Link','Headlines','News'])#,'Cbot_corn','CepeaR$_corn','CepeaU$','Rótulo'""",'Pais'"""])  
 
  http = urllib3.PoolManager()
 
  try:
    dados_paginas = http.request('GET',pagina)
 
  except HTTPError as e:
    print(e)
  except URLError:
    print("Server down or incorrect domain")  
   # Pegando os dados do html da pagina
  index = BeautifulSoup(dados_paginas.data,'lxml')
 
  #Procurando as tag "a" dentro da "div"
  links = index.find("div",{"id":"content"})
  links = links.find_all('a')
  #print(links)
  
  cont = 0
  base = pagina.split('/News')[0]
 
  #Percorrendo os link e salvando seu href(os links)    
  for link in links:    
    if cont == range(len(base)):        
      break;
 
    else:
      cont +=1   
        
    aux = str(link.get('href'))
    url = base + '/' + aux
 
    try:
      dados = http.request('GET',url)
    except:
     print("Pagina nao encontrada" )    
 
    #Pegando os dados do html do link acessado
    news = BeautifulSoup(dados.data,'lxml')
 
    #Pegando o conteúdo da Manchete
    conteudo = news.find("div",{"id": "content"})       
           
    #Pegando o conteudo das noticias
    noticias = news.find("div",{"id": "content"}) 
    noticia = news.find("div",{"id": "content"})
         
    if noticias != None:
      noticias = [y.get_text(strip=True) for y in news.find_all('p')]      
      noticias = str(noticias)
      noticias = noticias.replace("['",'')
      noticias = noticias.replace("']",'')
      noticias = noticias.replace('["','')
      noticias = noticias.replace('"]','')
      noticias = noticias.replace("', '",'') 
 
      conteudo = [x.get_text(strip=True) for x in news.find_all('h3')]
      conteudo = str(conteudo)
      conteudo = conteudo.replace("['",'')
      conteudo = conteudo.replace("']",'')
      conteudo = conteudo.replace('["','')
      conteudo = conteudo.replace('"]','')
      conteudo = conteudo.replace("', '",'')
       
      data = news.find({"h5"}).get_text(strip=True)
      if noticia != None:    
        noticia = [j for j in noticia.find_all('li',text=True)]      
        noticia = str(noticia)
        noticia = noticia.replace("['",'')
        noticia = noticia.replace("']",'')
        noticia = noticia.replace('["','')
        noticia = noticia.replace('"]','')
        noticia = noticia.replace("', '",'')
        noticia = noticia.replace('<"','')
        noticia = noticia.replace('">','') 
        noticia = noticia.replace('"\n','')
        noticia = re.sub('[\n,\t,<li>,/]','',noticia)
 
      data = news.find({"h5"}).get_text(strip=True)
 
      df = df.append({'Date': data,
                        'Link': url ,
                        'Headlines' : conteudo,
                         'News': noticias + noticia,
                          }, #'Pais': j},                                   
                          ignore_index=True)      
  
  return df

In [3]:
link2 = 'http://soybeansandcorn.com/News'

df = BuscaTexto(link2)
df.dropna(inplace=True)
df['Date'] = pd.to_datetime(df['Date'],errors='coerce')
df.dropna(subset=['Date'],inplace=True)


df.head(-1)

,Date,Headlines,Link,News
4,2021-05-14,Farmers in Mato Grosso have Sold 83% of their ...,http://soybeansandcorn.com/news/May14_21-Farme...,According to the Mato Grosso Institute of Agri...
5,2021-05-13,Conab Lowers Brazil Corn Production 2.5 mt to ...,http://soybeansandcorn.com/news/May13_21-Conab...,"In their May Crop Report, Conab lowered the 20..."
6,2021-05-13,Sugarcane Harvesting in Brazil during First Mo...,http://soybeansandcorn.com/news/May13_21-Sugar...,The Brazilian sugarcane production year goes f...
7,2021-05-12,2020/21 Brazil Safrinha Corn Production Contin...,http://soybeansandcorn.com/news/May12_21-20202...,The combination of historically late planting ...
8,2021-05-12,2020/21 Argentina Corn 22.7% Harvested vs. 32....,http://soybeansandcorn.com/news/May12_21-20202...,The later planted corn in Argentina benefited ...
...,...,...,...,...
2750,2014-01-08,Thousands of Additional Trucks Expected on Bra...,http://soybeansandcorn.com/news/Jan8_14-Thousd...,The combination of a record large soybean crop...
2751,2014-01-07,Brazilian Farmers Advised to stay Vigilant abo...,http://soybeansandcorn.com/news/Jan7_14-Brazil...,As farmers in Mato Grosso start to harvest som...
2752,2014-01-07,Livestock Producers in Brazil Fear Diversion o...,http://soybeansandcorn.com/news/Jan7_14-Livest...,Poultry and hog producers in southern Brazil a...
2753,2014-01-06,Capacity Expands at Latin America's Largest Gr...,http://soybeansandcorn.com/news/Jan6_14-Capaci...,America Latina Logistica (ALL) recently inaugu...


In [4]:
df.isnull().sum()

Date         0
Headlines    0
Link         0
News         0
dtype: int64

In [5]:
df.to_csv('Crawler_Corn_Soybeans.csv',index=False)

# Criando um DataFrame com os valores do Cbot, Cepea em real e Cepea  em dolar

In [6]:
!pip install OleFileIO_PL
!pip install scrapy

import scrapy
from scrapy.loader import ItemLoader
import requests
import OleFileIO_PL

     |████████████████████████████████| 122kB 7.0MB/s 
  Created wheel for olefile: filename=olefile-0.42.1-cp37-none-any.whl size=111385 sha256=4e1edb26bb9cbd41e6f9c03002d6a6e5de2b1888b76a4172cfac3cf9304fd426
  Stored in directory: /root/.cache/pip/wheels/7f/a6/17/86b00b33e526659bb900323f8563ccff99193af07457a31dd9
  ERROR: Failed building wheel for olefile
  Running setup.py clean for olefile
  ERROR: Failed cleaning build dir for olefile
Successfully built olefile
Failed to build olefile
     |████████████████████████████████| 256kB 7.6MB/s 
     |████████████████████████████████| 3.1MB 41.0MB/s 
     |████████████████████████████████| 61kB 10.2MB/s 
     |████████████████████████████████| 3.2MB 48.7MB/s 
     |████████████████████████████████| 3.2MB 28.1MB/s 
     |████████████████████████████████| 71kB 12.2MB/s 
     |████████████████████████████████| 256kB 41.7MB/s 
     |████████████████████████████████| 81kB 13.7MB/s 
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5

In [7]:

  
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning) 

http = urllib3.PoolManager()

pagina = 'https://www.cepea.esalq.usp.br/br/indicador/milho.aspx'
 
try:
  dados_paginas = http.request('GET',pagina)
 
except HTTPError as e:
  print(e)
except URLError:
  print("Server down or incorrect domain")  
   # Pegando os dados do html da pagina
index = BeautifulSoup(dados_paginas.data,'lxml')

  #Procurando as tag "a" dentro da "div"
links = index.find("div",{"class":"imagenet-links-after-table imagenet-col-2 imagenet-pa-l imagenet-bb imagenet-fl"})
links = links.find_all('a')[-1]
links = links.get('href')

print('\n\nClica no link para fazer o download: \n',links,'\n\n')

 
resp = requests.get(links)
ole = OleFileIO_PL.OleFileIO(resp.content) 

cepea = pd.read_excel(ole.openstream('Workbook'),skiprows=3)
cepea['Data'] = pd.to_datetime(cepea['Data'],errors='coerce')
cepea.rename(columns={'Data':'Date'},inplace=True)

cepea.head(-1)




Clica no link para fazer o download: 
 https://www.cepea.esalq.usp.br/br/indicador/series/milho.aspx?id=77 




,Date,À vista R$,À vista US$
0,2004-02-08,18.24,5.98
1,2004-03-08,18.04,5.91
2,2004-04-08,18.02,5.90
3,2004-05-08,18.06,5.89
4,2004-06-08,18.13,5.98
...,...,...,...
4171,2021-07-05,101.56,19.45
4172,2021-10-05,101.23,19.35
4173,2021-11-05,101.40,19.41
4174,2021-12-05,102.39,19.33


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
cbot = pd.read_csv('/content/drive/My Drive/dataset/cobot.csv', parse_dates=True, sep=';')
cbot.rename(columns={'date':'Date'},inplace=True)
cbot['Date'] = pd.to_datetime(cbot['Date'],errors='coerce')
cbot.dropna(inplace=True)
cbot.head()

cbot.head(-1)

,Date,value
0,1959-07-01,1.1770
1,1959-07-02,1.1760
2,1959-07-06,1.1710
3,1959-07-07,1.1710
4,1959-07-08,1.1700
...,...,...
15562,2021-04-09,5.7725
15563,2021-04-12,5.6900
15564,2021-04-13,5.8000
15565,2021-04-14,5.9400


In [10]:
dataset = pd.merge(df.reset_index(drop= True),cepea,  on='Date', how='left')
dataset.rename(columns={'À vista R$':'Cepea_R$'},inplace=True)
dataset.rename(columns={'À vista US$':'Cepea_US$'},inplace=True)

dataset.head(-1)

,Date,Headlines,Link,News,Cepea_R$,Cepea_US$
0,2021-05-14,Farmers in Mato Grosso have Sold 83% of their ...,http://soybeansandcorn.com/news/May14_21-Farme...,According to the Mato Grosso Institute of Agri...,101.34,19.23
1,2021-05-13,Conab Lowers Brazil Corn Production 2.5 mt to ...,http://soybeansandcorn.com/news/May13_21-Conab...,"In their May Crop Report, Conab lowered the 20...",101.79,19.21
2,2021-05-13,Sugarcane Harvesting in Brazil during First Mo...,http://soybeansandcorn.com/news/May13_21-Sugar...,The Brazilian sugarcane production year goes f...,101.79,19.21
3,2021-05-12,2020/21 Brazil Safrinha Corn Production Contin...,http://soybeansandcorn.com/news/May12_21-20202...,The combination of historically late planting ...,NaN,NaN
4,2021-05-12,2020/21 Argentina Corn 22.7% Harvested vs. 32....,http://soybeansandcorn.com/news/May12_21-20202...,The later planted corn in Argentina benefited ...,NaN,NaN
...,...,...,...,...,...,...
2737,2014-01-08,Thousands of Additional Trucks Expected on Bra...,http://soybeansandcorn.com/news/Jan8_14-Thousd...,The combination of a record large soybean crop...,22.93,10.14
2738,2014-01-07,Brazilian Farmers Advised to stay Vigilant abo...,http://soybeansandcorn.com/news/Jan7_14-Brazil...,As farmers in Mato Grosso start to harvest som...,24.46,11.09
2739,2014-01-07,Livestock Producers in Brazil Fear Diversion o...,http://soybeansandcorn.com/news/Jan7_14-Livest...,Poultry and hog producers in southern Brazil a...,24.46,11.09
2740,2014-01-06,Capacity Expands at Latin America's Largest Gr...,http://soybeansandcorn.com/news/Jan6_14-Capaci...,America Latina Logistica (ALL) recently inaugu...,NaN,NaN


In [11]:
dados = pd.merge(dataset.reset_index(drop=True),cbot, on='Date', how='left')
dados.rename(columns={'value':'Cbot_US$'},inplace=True)
dados.dropna(inplace=True)


dados.head()

,Date,Headlines,Link,News,Cepea_R$,Cepea_US$,Cbot_US$
32,2021-04-16,Rating of Safrinha Corn in Parana Declines due...,http://soybeansandcorn.com/news/Apr16_21-Ratin...,In their latest assessment of the safrinha cor...,97.88,17.52,5.855
33,2021-04-15,Corn-Based Ethanol Accounts for 9% of Brazil's...,http://soybeansandcorn.com/news/Apr15_21-Corn-...,The production of ethanol from corn continues ...,97.64,17.36,5.900
34,2021-04-15,Port of Paranagua Embarks Record Large Soybean...,http://soybeansandcorn.com/news/Apr15_21-Port-...,The Port of Paranagua in southern Brazil conti...,97.64,17.36,5.900
35,2021-04-14,IMEA Lowers Mato Grosso's Safrinha Corn Yield,http://soybeansandcorn.com/news/Apr14_21-IMEA-...,After the completion of the safrinha corn plan...,96.92,17.08,5.940
36,2021-04-14,"Argentina Soybeans 3.6% Harvested, Early Yield...",http://soybeansandcorn.com/news/Apr14_21-Argen...,There were good rains across central and south...,96.92,17.08,5.940


## || Criando colunas com o Tarjet de cada valor

In [15]:
def Tarjet(x, y):
  if x >= y and ((x - y) * 100) / y:
    return int(1)
  
  elif y >= x and ((y - x) * 100) / y:
    return 0

  else:
    return 0
  

In [16]:
dados['TARJET_CEPEA_R$'] =[np.nan for x in range(len(dados['Date']))]
dados['TARJET_CEPEA_US$'] =[np.nan for x in range(len(dados['Date']))]
dados['TARJET_CBOT_US$'] =[np.nan for x in range(len(dados['Date']))]

for ini in range(len(dados)):
  if ini >= float(0.05):
    antes_real = float(dados['Cepea_R$'].iloc[ini -1])
    agora_real = float(dados['Cepea_R$'].iloc[ini])
   
    antes_dolar = float(dados['Cepea_US$'].iloc[ini -1])
    agora_dolar = float(dados['Cepea_US$'].iloc[ini])
    
    antes_cbot = float(dados['Cbot_US$'].iloc[ini -1])
    agora_cbot = float(dados['Cbot_US$'].iloc[ini])
    
    dados['TARJET_CEPEA_R$'].iloc[ini - 1] = Tarjet( antes_real, agora_real)
    dados['TARJET_CEPEA_US$'].iloc[ini - 1] = Tarjet(antes_dolar, agora_dolar)
    dados['TARJET_CBOT_US$'].iloc[ini - 1] = Tarjet(antes_cbot, agora_cbot)


  
  


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [17]:
dados.head(-1)



,Date,Headlines,Link,News,Cepea_R$,Cepea_US$,Cbot_US$,TARJET_CEPEA_R$,TARJET_CEPEA_US$,TARJET_CBOT_US$
32,2021-04-16,Rating of Safrinha Corn in Parana Declines due...,http://soybeansandcorn.com/news/Apr16_21-Ratin...,In their latest assessment of the safrinha cor...,97.88,17.52,5.855,1.0,1.0,0.0
33,2021-04-15,Corn-Based Ethanol Accounts for 9% of Brazil's...,http://soybeansandcorn.com/news/Apr15_21-Corn-...,The production of ethanol from corn continues ...,97.64,17.36,5.900,0.0,0.0,0.0
34,2021-04-15,Port of Paranagua Embarks Record Large Soybean...,http://soybeansandcorn.com/news/Apr15_21-Port-...,The Port of Paranagua in southern Brazil conti...,97.64,17.36,5.900,1.0,1.0,0.0
35,2021-04-14,IMEA Lowers Mato Grosso's Safrinha Corn Yield,http://soybeansandcorn.com/news/Apr14_21-IMEA-...,After the completion of the safrinha corn plan...,96.92,17.08,5.940,0.0,0.0,0.0
36,2021-04-14,"Argentina Soybeans 3.6% Harvested, Early Yield...",http://soybeansandcorn.com/news/Apr14_21-Argen...,There were good rains across central and south...,96.92,17.08,5.940,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...
2734,2014-01-14,Reaction to Last Week's USDA Reports,http://soybeansandcorn.com/news/Jan14_14-React...,USDA Crop Summary 2013- The biggest surprise i...,27.35,11.61,4.315,1.0,1.0,1.0
2735,2014-01-09,Brazilian Corn Acreage Losing out to Higher Pr...,http://soybeansandcorn.com/news/Jan9_14-Brazii...,Farmers all across Brazil opted for more soybe...,22.51,10.04,4.120,0.0,0.0,0.0
2736,2014-01-08,"Harvest Picks Up in Mato Grosso, Dry Concerns ...",http://soybeansandcorn.com/news/Jan8_14-Harves...,Farmers in Mato Grosso are starting to harvest...,22.93,10.14,4.170,0.0,0.0,0.0
2737,2014-01-08,Thousands of Additional Trucks Expected on Bra...,http://soybeansandcorn.com/news/Jan8_14-Thousd...,The combination of a record large soybean crop...,22.93,10.14,4.170,0.0,0.0,0.0


In [ ]:
dados.to_csv('Data_Tarjet.csv')